Assignment 1 - Group 2

Exercise 1

a)In the interactive session, we saw reaction fluxes in a linear pathway being equal to each other due to mass balance constraints. Do you observe the same thing now for the maximal reaction activities? Explain your observations.

What we observed is that maximal reaction activities were generally not equal. The values in the pathway vary significantly. This happens beacuse the maximal activity is the maximum rate a reaction can have. This is determined by the concentration of the enzymes that catalyses the reactions. This is deifferent from what we saw with a linear pathway where all reactions must be equal to maintain the mass balance.

b)Some reactions have no maximal reaction activity data. Identify at least two different kinds of such reactions, and explain why gene expression-derived data would not be applicable for these kinds of reactions

- Transport reactions: are not directly catalyzed by a single enzyme or gene product. Depend on other properties like concentration gradients which makes gene expression-derived data irrelevant.

- Non-enzymatic reactions: occur randomly without an enzyme. Depend on chemical properties of the involved metabolites so we have no genes. An example can be EX_fum_e.



Exercise 2

In [ ]:
import cobra
import pandas as pd

In [6]:
model = cobra.io.load_json_model('e_coli_core.json')
df = pd.read_csv('e_coli_core_expression.csv')

In [8]:
mapping = dict(zip(df["# Reaction ID"], df[" reaction activity [mmol/gDW/h] "]))

In [9]:
#Reversible/Irreversible reactions, lower and upper bounds
DEFAULT_HIGH = 1000.0
changed = []
unchanged = []

for reaction in model.reactions:
    #glucose exchange reaction
    if reaction.id == "EX_glc__D_e":
        reaction.lower_bound = -DEFAULT_HIGH
        reaction.upper_bound =  DEFAULT_HIGH
        changed.append((reaction.id, "special: EX_glc__D_e set to ±1000"))
        continue

    #ATP maintenance
    if reaction.id == "ATPM":
        if reaction.id in mapping:
            val = mapping[reaction.id]
            reaction.upper_bound = val
            changed.append((reaction.id, f"ATPM: lower preserved, upper set to {val}"))
        else:
            unchanged.append(reaction.id)
        continue

    if reaction.id in mapping:
        val = float(mapping[reaction.id])
        if reaction.lower_bound < 0:
            #reversible reaction
            reaction.lower_bound = -val
            reaction.upper_bound =  val
            changed.append((reaction.id, f"reversible set to ±{val}"))
        else:
            #irreversible reaction
            reaction.lower_bound = 0.0
            reaction.upper_bound =  val
            changed.append((reaction.id, f"irreversible set to [0, {val}]"))
    else:
        #no data
        unchanged.append(reaction.id)

In [10]:
rows = []
for reaction in model.reactions:
    rows.append((reaction.id, reaction.lower_bound, reaction.upper_bound))

for r in rows[:20]:
    print(f"{r[0]}\t{r[1]}\t{r[2]}")

PFK	0.0	12.12
PFL	0.0	1.0
PGI	-13.12	13.12
PGK	-23.13	23.13
PGL	0.0	8.12
ACALD	-1.16	1.16
AKGt2r	-3.1	3.1
PGM	-20.01	20.01
PIt2r	-6.03	6.03
ALCD2x	-9.01	9.01
ACALDt	-2.29	2.29
ACKr	-1.19	1.19
PPC	0.0	2.56
ACONTa	-25.35	25.35
ACONTb	-25.35	25.35
ATPM	8.39	1000.0
PPCK	0.0	25.23
ACt2r	-3.23	3.23
PPS	0.0	2.5
ADK1	-30.57	30.57
